In [1]:
import sys
sys.path.append('../')

In [2]:
from sklearn import datasets
from sklearn.metrics import accuracy_score
import pymlearn.linear
import numpy as np

[Warning] The derivatives are not implemented in fully vectorized way.


/home/mkopersk/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [3]:
reload(pymlearn.linear)

<module 'pymlearn.linear' from '../pymlearn/linear.pyc'>

In [4]:
data_set = datasets.load_digits(10)
test_set_size = data_set['target'].shape[0] / 3
Ytr = data_set['target'][test_set_size:]
Xtr = data_set['data'][test_set_size:]
validation_set_size = Xtr.shape[0] / 3
# Xvd = Xtr[:validation_set_size]
# Yvd = Ytr[:validation_set_size]
# Xtr = Xtr[validation_set_size:]
# Ytr = Ytr[validation_set_size:]
Yts = data_set['target'][:test_set_size]
Xts = data_set['data'][:test_set_size]

In [5]:
%%time
p_lambda = 1
W = pymlearn.linear.train_classifer(Xtr, Ytr, loss='softmax', reg=p_lambda, solver='GD')

CPU times: user 37.6 s, sys: 898 ms, total: 38.5 s
Wall time: 4.78 s


In [5]:
# W = W.keys()[0].get_value()
W = W.get_value()

In [6]:
W = W.reshape(10,-1)
print 'Accuracy on train: %0.2f%%' % (accuracy_score(Ytr, pymlearn.linear.classify(W, Xtr)) * 100)
print 'Accuracy on test: %0.2f%%' % (accuracy_score(Yts, pymlearn.linear.classify(W, Xts)) * 100)

Accuracy on train: 96.16%
Accuracy on test: 91.82%


array([ 0.38461349,  0.54111484,  0.30777763,  0.29831978,  0.70138548,
        0.16032543,  0.28091796,  0.06723132,  0.41531958,  0.00936577,
        0.49287225,  0.17751858,  0.61225323,  0.11229339,  0.75713989,
        0.72958567,  0.69822749,  0.03293972,  0.88670999,  0.52954358,
        0.2047859 ,  0.39220344,  0.95367108,  0.79782938,  0.79187533,
        0.46979735,  0.82412274,  0.59995856,  0.41358166,  0.6466326 ,
        0.31824963,  0.60498499,  0.05072902,  0.01531144,  0.18921635,
        0.02951514,  0.1530959 ,  0.95121525,  0.69496635,  0.55289587,
        0.91547142,  0.34546004,  0.1820328 ,  0.53971596,  0.44840406,
        0.02931197,  0.07733575,  0.69563447,  0.97969947,  0.4475829 ,
        0.67590469,  0.51312345,  0.99754657,  0.93610698,  0.32047171,
        0.77078826,  0.80578302,  0.85969954,  0.04284026,  0.37283109,
        0.65019889,  0.05191589,  0.66987585,  0.20572214,  0.7336652 ,
        0.56952808,  0.88077174,  0.89532144,  0.71793791,  0.92

In [31]:
p_lambda = 0.2
clf_ = pymlearn.linear.LinearClassifer(reg=p_lambda, loss='softmax')
clf_.fit(Xtr, Ytr)
print 'Accuracy on train: %0.2f%%' % (accuracy_score(Ytr, clf_.predict(Xtr)) * 100)
print 'Accuracy on test: %0.2f%%' % (accuracy_score(Yts, clf_.predict(Xts)) * 100)

Accuracy on train: 98.16%
Accuracy on test: 93.32%


In [10]:
import sklearn.grid_search

params_grid = {'loss': ['hinge', 'softmax'], 'penalty': ['L1', 'L2'], 
                                                 'reg': np.logspace(-2,2,10)}
estimator = pymlearn.linear.LinearClassifer()
clf_ = sklearn.grid_search.GridSearchCV(estimator, params_grid, cv=[(np.arange(validation_set_size, Xtr.shape[0]), 
                                                                    np.arange(validation_set_size))], n_jobs=4)
clf_.fit(Xtr, Ytr)

print 'Accuracy on train: %0.2f%%' % (accuracy_score(Ytr, clf_.predict(Xtr)) * 100)
print 'Accuracy on test: %0.2f%%' % (accuracy_score(Yts, clf_.predict(Xts)) * 100)

Accuracy on train: 98.00%
Accuracy on test: 91.82%


In [11]:
clf_.best_score_

0.9172932330827067

In [12]:
clf_.best_params_

{'loss': 'hinge', 'penalty': 'L1', 'reg': 0.077426368268112694}

In [24]:
%pylab
from scipy.interpolate import interp1d

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


/home/mkopersk/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['clf']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [33]:
W = clf_.params
for i in range(W.shape[0]):
    ax = plt.subplot(2, 5, i+1)
    v = W[i,:-1].reshape((8,8))
    ax.matshow(v, cmap=plt.cm.gray)

In [483]:
W_s = W.copy()
W_s[W_s < 1e-2] = 0
W_orig = W.copy()
W = W_s